In [1]:
f=open('intermediate.tsv')
content=f.read()

In [23]:
forms=[]
lemmas=[]
cpostags=[]
postags=[]
feats=[]
heads=[]
deprels=[]

for elem in content.split("\n"):
    if(elem!=""):
        print(elem)
        forms.append(elem.split("\t")[0].decode('utf-8'))
        lemmas.append(elem.split("\t")[1].decode('utf-8'))
        cpostags.append(elem.split("\t")[2].decode('utf-8'))
        postags.append(elem.split("\t")[3].decode('utf-8'))
        feats.append(elem.split("\t")[4].decode('utf-8'))
        heads.append(int(elem.split("\t")[5]))
        deprels.append(elem.split("\t")[6].decode('utf-8'))
    

Barack	Barack	PROPN	NNP	NounType=prop|Number=sing	4	compound	__
Hussein	Hussein	PROPN	NNP	NounType=prop|Number=sing	4	compound	__
Obama	Obama	PROPN	NNP	NounType=prop|Number=sing	4	compound	__
II	II	PROPN	NNP	NounType=prop|Number=sing	12	nsubj	__
(	(	PUNCT	-LRB-	PunctType=brck|PunctSide=ini	4	punct	_SpaceAfter=No
born	bear	VERB	VBN	VerbForm=part|Tense=past|Aspect=perf	4	acl	__
August	August	PROPN	NNP	NounType=prop|Number=sing	6	npadvmod	__
4	4	NUM	CD	NumType=card	7	nummod	_SpaceAfter=No
,	,	PUNCT	,	PunctType=comm	7	punct	__
1961	1961	NUM	CD	NumType=card	7	nummod	_SpaceAfter=No
)	)	PUNCT	-RRB-	PunctType=brck|PunctSide=fin	4	punct	__
is	be	AUX	VBZ	VerbForm=fin|Tense=pres|Number=sing|Person=three	0	ROOT	__
an	an	DET	DT	_	15	det	__
American	american	ADJ	JJ	Degree=pos	15	amod	__
politician	politician	NOUN	NN	Number=sing	12	attr	__
who	who	PRON	WP	_	17	nsubj	__
is	be	AUX	VBZ	VerbForm=fin|Tense=pres|Number=sing|Person=three	15	relcl	__
the	the	DET	DT	_	22	det	__
44th	44th	ADJ	JJ	Degree=pos	22	

In [7]:
from dep_tregex import Tree
tree = Tree(forms, lemmas, cpostags, postags, feats, heads, deprels)

In [40]:
forms[5]

u'born'

In [41]:
from dep_tregex import parse_pattern
#pattern = parse_pattern("x lemma 'be' and > y deprel 'NSUBJ'")
pattern = parse_pattern("x deprel 'nsubj'")
matchings, backrefs_map = [], {}
for node in range(1, len(tree) + 1):
    if pattern.match(tree, node, backrefs_map):
        matchings.append(node)

In [42]:
matchings

[4, 16]

In [43]:
import dep_tregex
html_file = open('tree.html', 'w')
dep_tregex.write_prologue_html(html_file)
#dep_tregex.write_tree_html(file, html_file)
dep_tregex.write_epilogue_html(html_file)

In [44]:
import os
#Set standford parser and models in your environment variables.
from nltk.parse import CoreNLPParser
from nltk.tree import ParentedTree, Tree
parser = CoreNLPParser(url='http://localhost:9000')

In [45]:
class TripleExtraction(object):

    VERBS = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    NOUNS = ['NN', 'NNS', 'NNP', 'NNPS']

    def __init__(self, filepath_to_conll=None):
        self.filepath_to_conll = None
        self.tokenlist = None
        self.tokenTree = None
        if filepath_to_conll:
            self.filepath_to_conll = filepath_to_conll
            data_file = open(filepath_to_conll, "r", encoding="utf-8")
            self.tokenlist = parse_single(data_file)
            self.tokenTree = tokenlist[0].to_tree()


        self.treegex_patterns = [

        ]

    def treegex(self, sentence):
        '''
        make sure the coreNLP server is running
        the default url is set to - http://localhost:9000/tregex, if this is not the url which you are using set the required url
        in the param url, and pass it to the treegex_api function (third parameter)
        '''

        responses = treegex_api(self.treegex_patterns, sentence)

    def find_subject(self, t): 
        for s in t.subtrees(lambda t: t.label() == 'NP'):
            for n in s.subtrees(lambda n: n.label().startswith('NN')):
                return (n[0], self.find_attrs(n))
                # return n[0]
                
    def find_predicate(self, t):    
        v = None
        
        for s in t.subtrees(lambda t: t.label() == 'VP'):
            for n in s.subtrees(lambda n: n.label().startswith('VB')):
                v = n
            return (v[0], self.find_attrs(v))
            # return v[0]
        
    def find_object(self,t):    
        for s in t.subtrees(lambda t: t.label() == 'VP'):
            for n in s.subtrees(lambda n: n.label() in ['NP', 'PP', 'ADJP']):
                if n.label() in ['NP', 'PP']:
                    for c in n.subtrees(lambda c: c.label().startswith('NN')):
                        return (c[0], self.find_attrs(c))
                        # return c[0]
                else:
                    for c in n.subtrees(lambda c: c.label().startswith('JJ')):
                        return (c[0], self.find_attrs(c))
                        # return c[0]
                    
    def find_attrs(self, node):
        attrs = []
        p = node.parent()
        
        # Search siblings
        if node.label().startswith('JJ'):
            for s in p:
                if s.label() == 'RB':
                    attrs.append(s[0])
                    
        elif node.label().startswith('NN'):
            for s in p:
                if s.label() in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP']:
                    attrs.append(' '.join(s.flatten()))
        
        elif node.label().startswith('VB'):
            for s in p:
                if s.label() == 'ADVP':
                    attrs.append(' '.join(s.flatten()))
                    
        # Search uncles
        if node.label().startswith('JJ') or node.label().startswith('NN'):
            for s in p.parent():
                if s != p and s.label() == 'PP':
                    attrs.append(' '.join(s.flatten()))
                    
        elif node.label().startswith('VB'):
            for s in p.parent():
                if s != p and s.label().startswith('VB'):
                    attrs.append(s[0])
                    
        return attrs

    def main(self, sentence):
        try:
            subject = self.find_subject(sentence)
            predicate = self.find_predicate(sentence)
            object_ = self.find_object(sentence)
            # print("triplet - ", subject, predicate, object_)
            return (subject, predicate, object_)
        except:
            return ()

    def treebank(self, sentence):
        tree = list(parser.raw_parse(sentence))[0]
        triple = self.main(ParentedTree.convert(tree))
        return triple

if __name__=="__main__" :
    import sys   
    # Parse the example sentence
    sent = "Astatine is a radioactive chemical element with the chemical symbol At and atomic number 85, and is the rarest naturally occurring element on the Earth's crust."
    t = TripleExtraction()
    t.treebank(sent)

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22tokenize.whitespace%22%3A+%22false%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%2C+%22ssplit.ssplit.eolonly%22%3A+%22true%22%7D (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd858e437d0>: Failed to establish a new connection: [Errno 111] Connection refused',))